In [1]:
class Scalar():
    def __init__(self, data: int, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        
    def __add__(self, other: 'Scalar') -> 'Scalar':
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar((self.data + other.data), (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data - other.data, (self, other), '-')
    
        def _backward():
            # d(self - other)/dself = +1
            # d(self - other)/dother = -1
            self.grad += 1 * out.grad
            other.grad += -1 * out.grad
    
        out._backward = _backward
        return out
    
    def __rsub__(self, other):
        # handles cases like 5 - Scalar(3)
        other = other if isinstance(other, Scalar) else Scalar(other)
        return other - self

    
    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar((self.data * other.data), (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def relu(self):
        out = Scalar(max(0, self.data), (self,), 'relu')
        def _backward():
            self.grad += out.grad * (1 if self.data > 0 else 0)
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1
        for v in reversed(topo):
            v._backward()
            
    def zero_grad(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
    
        for v in topo:
            v.grad = 0

    def __repr__(self):
        return f"Scalar(data={self.data}, grad={self.grad}{f', {self._op}' if len(self._op) > 0 else ''})"

In [2]:
x = Scalar(2)
y = x * 2
z = x + y
y, x, z

(Scalar(data=4, grad=0, *), Scalar(data=2, grad=0), Scalar(data=6, grad=0, +))

In [3]:
z.backward()
x, y, z

(Scalar(data=2, grad=3), Scalar(data=4, grad=1, *), Scalar(data=6, grad=1, +))

In [4]:
z.zero_grad()
x, y, z

(Scalar(data=2, grad=0), Scalar(data=4, grad=0, *), Scalar(data=6, grad=0, +))

In [5]:
import torch

In [6]:
xt, yt = torch.tensor(2., requires_grad=True), torch.tensor(5., requires_grad=True)
zt = xt + yt
xt, yt, zt

(tensor(2., requires_grad=True),
 tensor(5., requires_grad=True),
 tensor(7., grad_fn=<AddBackward0>))

In [7]:
zt.backward()
xt.grad, yt.grad

(tensor(1.), tensor(1.))

In [13]:
import random
class Neuron:
    def __init__(self, nin):
        import random
        self.w = [Scalar(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Scalar(0)

    def __call__(self, x):
        # x is list of Scalars
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.relu()
        return out

    def parameters(self):
        return self.w + [self.b]


In [14]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        # Call each neuron
        outs = [n(x) for n in self.neurons]
        # If this is the last layer, just return list
        return outs
    
    def parameters(self):
        # Flatten all neuron parameters
        return [p for n in self.neurons for p in n.parameters()]


In [15]:
class MLP:
    def __init__(self, nin, nouts):
        # nouts = list like [16, 16, 1]
        sizes = [nin] + nouts
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(nouts))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x[0] if len(x) == 1 else x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [16]:
# Data: XOR truth table
xs = [
    [Scalar(0), Scalar(0)],
    [Scalar(0), Scalar(1)],
    [Scalar(1), Scalar(0)],
    [Scalar(1), Scalar(1)]
]
ys = [Scalar(0), Scalar(1), Scalar(1), Scalar(0)]

# Build network: 2 inputs, two hidden layers (4 each), 1 output
net = MLP(2, [4, 4, 1])

# Training
for k in range(100):
    # Forward
    ypred = [net(x) for x in xs]
    loss = sum((yout - ygt) * (yout - ygt) for ygt, yout in zip(ys, ypred))

    # Zero gradients
    for p in net.parameters():
        p.grad = 0
    loss.backward()

    # Simple SGD step
    for p in net.parameters():
        p.data -= 0.1 * p.grad

    if k % 10 == 0:
        print(f"step {k}: loss = {loss.data:.4f}")


step 0: loss = 1.3365
step 10: loss = 0.1020
step 20: loss = 0.0009
step 30: loss = 0.0000
step 40: loss = 0.0000
step 50: loss = 0.0000
step 60: loss = 0.0000
step 70: loss = 0.0000
step 80: loss = 0.0000
step 90: loss = 0.0000


In [17]:
for x in xs:
    print([xi.data for xi in x], '->', net(x).data)

[0, 0] -> 1.898219760812156e-07
[0, 1] -> 0.9999999681630977
[1, 0] -> 0.9999999733241657
[1, 1] -> 0
